In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import os
from PIL import Image
import kaggle
from sklearn.model_selection import train_test_split
import pickle # to save training history

In [4]:
data_dir = './human-face-emotions/data'

In [ ]:
full_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode="int",
    image_size=(48, 48),
    color_mode="grayscale",
    batch_size=None,
    shuffle=True,
    seed=42
)

class_names = full_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)

Found 57756 files belonging to 5 classes.
Classes: ['Angry', 'Fear', 'Happy', 'Sad', 'Suprise']


In [6]:
full_data = list(full_ds.as_numpy_iterator())

images = [x[0] for x in full_data]  # list of arrays
labels = [x[1] for x in full_data]  # list of ints

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(
    images, labels,
    test_size=0.30,         # 30% val+test
    random_state=42,
    stratify=labels 
)

x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp,
    test_size=0.50,         # 15% val, 15% test
    random_state=42,
    stratify=y_temp
)

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds   = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [9]:
batch_size = 128

train_ds = train_ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# early stopping to prevent overfitting
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

In [17]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.03),
    tf.keras.layers.RandomZoom(0.05)
])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(48, 48, 1)),
    data_augmentation,

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

c:\Users\sandralm\AppData\Local\miniconda3\envs\IDS\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,269 (2.61 MB)

 Trainable params: 683,269 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history_e2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[callback]
)

Epoch 1/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 19s 56ms/step - accuracy: 0.3004 - loss: 5.3961 - val_accuracy: 0.4209 - val_loss: 1.3722
Epoch 2/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step - accuracy: 0.4149 - loss: 1.4000 - val_accuracy: 0.4606 - val_loss: 1.2731
Epoch 3/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.4534 - loss: 1.3059 - val_accuracy: 0.5017 - val_loss: 1.1876
Epoch 4/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 25s 79ms/step - accuracy: 0.4912 - loss: 1.2345 - val_accuracy: 0.5316 - val_loss: 1.1176
Epoch 5/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 28s 89ms/step - accuracy: 0.5096 - loss: 1.1923 - val_accuracy: 0.5570 - val_loss: 1.1087
Epoch 6/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.5404 - loss: 1.1411 - val_accuracy: 0.5734 - val_loss: 1.0550
Epoch 7/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.5428 - loss: 1.1162 - val_accuracy: 0.5877 - val_loss: 1.0403
Epoch 8/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 25s 78ms/step - accuracy: 0.5600 - loss: 1.0950 - 

In [14]:
test_loss, test_acc = model.evaluate(test_ds)
print("Final Test Accuracy:", test_acc)

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6367 - loss: 0.9056
Final Test Accuracy: 0.6298476457595825


In [ ]:
model.save('models/cnn_augmentation2.keras')

In [ ]:
with open("training_history/history_cnn_augmentation2.pkl", "wb") as f:
    pickle.dump(history_e2.history, f)

### No horisontal flip

In [23]:
data_augmentation_2 = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.03),
    tf.keras.layers.RandomZoom(0.05)
])

In [ ]:
model_aug3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(48, 48, 1)),
    data_augmentation_2,

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_aug3.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_aug3.summary()

c:\Users\sandralm\AppData\Local\miniconda3\envs\IDS\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,269 (2.61 MB)

 Trainable params: 683,269 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
history_aug3 = model_aug3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[callback]
)

Epoch 1/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 38s 114ms/step - accuracy: 0.2972 - loss: 3.3995 - val_accuracy: 0.4154 - val_loss: 1.3619
Epoch 2/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 25s 78ms/step - accuracy: 0.4234 - loss: 1.3676 - val_accuracy: 0.4885 - val_loss: 1.2166
Epoch 3/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.4793 - loss: 1.2553 - val_accuracy: 0.5470 - val_loss: 1.1115
Epoch 4/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 24s 75ms/step - accuracy: 0.5105 - loss: 1.1852 - val_accuracy: 0.5656 - val_loss: 1.0907
Epoch 5/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 24s 77ms/step - accuracy: 0.5359 - loss: 1.1305 - val_accuracy: 0.5877 - val_loss: 1.0231
Epoch 6/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 24s 77ms/step - accuracy: 0.5581 - loss: 1.0853 - val_accuracy: 0.5867 - val_loss: 1.0260
Epoch 7/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 23s 74ms/step - accuracy: 0.5681 - loss: 1.0590 - val_accuracy: 0.6087 - val_loss: 0.9924
Epoch 8/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 24s 77ms/step - accuracy: 0.5935 - loss: 1.0078 -

In [26]:
test_loss, test_acc = model_aug3.evaluate(test_ds)
print("Final Test Accuracy:", test_acc)

68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7396 - loss: 0.7225
Final Test Accuracy: 0.730840265750885


In [ ]:
with open("training_history/history_cnn_augmentation3.pkl", "wb") as f:
    pickle.dump(history_aug3.history, f)